# PHI augmentation : i2b2-2006 dataset processing

The i2b2 2014 de-identification dataset can be accessed from https://portal.dbmi.hms.harvard.edu.

The data processing mainly refers the guidance to this link:
https://github.com/juand-r/entity-recognition-datasets/tree/master/data/i2b2_2006

1.Combining all the XML files, then randomly splitting the records into train, validation, test set with the proportion of 7:1:2.

2.Follow the guidance from above link to convert records from XML fromat to CONLL format.

3.Double check the sentences in file should be seperated by a single empty line.

4.Convert records from CONLL fromat to BIO format.

In [3]:
file_path = 'C:/Users/zhou/Desktop/i2b2_2006_train.xml.conll'
file_path2 = 'C:/Users/zhou/Desktop/i2b2_2006_train_bio.txt'

# The sentences in input file should be seperated by a single empty line.
# CONLL fromat -->  BIO format

reserve_content = []
with open(file_path, 'r') as f_wr:
    content = f_wr.readlines()
    for row in range(1, len(content)):
        new_line = ''
        new_line2 = ''
        
        if len(content[row]) == 1:  # cope with empty line
        	# print('this a non ')
        	next_line = '\n'
        	reserve_content.append(next_line)
        else:
            all_token_Previous = content[row-1].split()
            all_token = content[row].split()
      
            if len(content[row-1]) != 1 and all_token[-1] != 'O':   # non-empty line & non-empty in previous line & this label not 'O'
                if all_token[-1] != all_token_Previous[-1]:  # whether these two lines have same label 
                    new_label = 'B-' + all_token[-1]
                if  all_token[-1] == all_token_Previous[-1]: 
            	    new_label = 'I-' + all_token[-1]
                all_token[-1] = new_label
                for i in all_token:
                    new_line = new_line+' '+ i
                new_line = new_line+'\n'
                reserve_content.append(new_line)

            if len(content[row-1]) != 1 and all_token[-1] == 'O':  # non-empty line & non-empty in previous line & this label is 'O'
                    for i in all_token:
                        new_line2 = new_line2+' '+ i
                    new_line2 = new_line2 + '\n'
                    reserve_content.append(new_line2)

            elif len(content[row-1]) == 1:   # non-empty line & empty line in previous
                if all_token[-1] == 'O':   # this label is 'O'
                    for i in all_token:
                        new_line2 = new_line2+' '+ i
                    new_line2 = new_line2 + '\n'
                    reserve_content.append(new_line2)
                else:      # this label not 'O'
                    new_label = 'B-' + all_token[-1]
                    all_token[-1] = new_label
                    for i in all_token:
                        new_line2 = new_line2+' '+ i
                    new_line2 = new_line2 + '\n'
                    reserve_content.append(new_line2)

with open(file_path2, 'w') as f_w:
    for line in reserve_content:
        f_w.write(line)

5.Replace the label of low frequency PHI type, namely 'B-AGE' and 'I-AGE', with 'O'.

If want to conduct PHI augmentation, run the step 6,7; If not, skip to step 7.

6.Conduct PHI augmentation, run the following code.
Notice that we replace PHI entity in fine-grainity (in PHI type level).

In [9]:
import random

file_path_write = 'C:/Users/zhou/Desktop/1.all_PHI_from_internet/'
labels_in_2006 = ['PATIENT', 'DOCTOR', 'HOSPITAL', 'ID', 'DATE', 'LOCATION', 'PHONE']
label_content = [[] for i in range(len(labels_in_2006))]
for k in range(len(labels_in_2006)):
    file_path = file_path_write + labels_in_2006[k] + '.txt'
    with open(file_path, 'r', encoding='utf-8') as f_w:
        content = f_w.readlines()
        for j in content:
            label_content[k].append(j.strip())


def replace_PHI(label, line_number):
    preserve_list = []
    if label in labels_in_2006:
        label_index = labels_in_2006.index(label)
        words_list = label_content[label_index]

        random.seed(global_seed)  # every epoch, it produces different dataset, also reimplementable.
        random.shuffle(words_list)
        words_num = len(words_list)

        random.seed(line_number)  # line_number is defined for specific PHI instance
        random_index = random.randint(0, words_num - 1)
        random_w = words_list[random_index].strip()

        words_num = random_w.split()
        if len(words_num) == 1:
            new_line = random_w+'   '+'B-'+label
            preserve_list.append(new_line)
        if len(words_num) > 1:
            for i in range(len(words_num)):
                if i == 0:
                    new_line = words_num[i]+'   '+'B-'+label
                    preserve_list.append(new_line)
                if i > 0:
                    new_line = words_num[i]+'   '+'I-'+label
                    preserve_list.append(new_line)
    return preserve_list


file_path_2014 = 'C:/Users/zhou/Desktop/i2b2_2006_train_bio.txt'
global_seed = 2
repalced_file_path = 'C:/Users/zhou/Desktop/new_replaced_2006_train_seed'+str(global_seed)+'.txt'


reserve_content = []
with open(file_path_2014, 'r', encoding='utf-8') as f_wr:
    content = f_wr.readlines()
    for row in range(0, len(content)-1):
        if len(content[row]) == 1:  # cope with empty line
            reserve_content.append('')
        if len(content[row]) != 1 and len(content[row+1]) != 1:
            all_token_Next = content[row + 1].split()
            all_token = content[row].split()
            this_line_label = all_token[-1]
            next_line_label = all_token_Next[-1]
            if this_line_label.startswith('O'):
                new_line = all_token[0]+'   '+all_token[-1]
                reserve_content.append(new_line)
                continue
            if this_line_label.startswith('B-') and next_line_label.startswith('I-'):
                continue
            if this_line_label.startswith('B-') and not next_line_label.startswith('I-'):
                label = this_line_label.split('-')[-1]
                new_words = replace_PHI(label, row)
                for k in new_words:
                    reserve_content.append(k)
            if this_line_label.startswith('I-') and next_line_label.startswith('I-'):
                continue
            if this_line_label.startswith('I-') and not next_line_label.startswith('I-'):
                label = this_line_label.split('-')[-1]
                # print(label)
                new_words = replace_PHI(label, row)
                # print(new_words)
                for k in new_words:
                    reserve_content.append(k)

        if len(content[row]) != 1 and len(content[row+1]) == 1:
            all_token = content[row].split()
            this_line_label = all_token[-1]
            if this_line_label.startswith('B-') or this_line_label.startswith('I-'):
                label = this_line_label.split('-')[-1]
                new_words = replace_PHI(label, row)
                for k in new_words:
                    reserve_content.append(k)
            if this_line_label.startswith('O'):
                new_line = all_token[0]+'   '+all_token[-1]
                reserve_content.append(new_line)
                continue


with open(repalced_file_path, 'w', encoding='utf-8') as f_w:
    for line in reserve_content:
        f_w.writelines(line+'\n')

7.Combine PHI into five PHI categories.

In [7]:
label_contact = 'PHONE'
label_location = ['LOCATION', 'HOSPITAL']
label_name = ['PATIENT', 'DOCTOR']

reserve_content = []
file_path = 'C:/Users/zhou/Desktop/new_replaced_2006_train_seed2.txt'
file_path_write = 'C:/Users/zhou/Desktop/2006_train.txt'

with open(file_path, 'r', encoding='utf-8') as f_wr:
    content = f_wr.readlines()
    for line in content:
        if len(line.split()) == 0 :  # preserve empty line
            reserve_content.append(line)
        else:
            words = line.split()
            label = words[-1]
            if label == 'O':
                reserve_content.append(line)
            else:
                label_type = label.split('-')[1]
                if label_type in label_location:
                    new_label = label.replace(label_type, 'LOCATION')  # B-HOSPITAL --> B-LOCATION
                    new_line = line.replace(label, new_label)  # XXX B-LOCATION
                    reserve_content.append(new_line)
                elif label_type == label_contact:
                    line = line.replace(label_type,'CONTACT')
                    reserve_content.append(line)
                elif label_type in label_name:
                    new_label = label.replace(label_type, 'NAME')
                    new_line = line.replace(label, new_label) 
                    reserve_content.append(new_line)
                else:
                    assert label_type in ['DATE', 'ID']
                    reserve_content.append(line)

with open(file_path_write, 'w', encoding='utf-8') as f_w:
    for line in reserve_content:
        f_w.write(line)

8.If want to conduct context augmentation, turn to relevant step.